**In this tutorial of the [Titanic Use Case](https://www.kaggle.com/c/titanic), you will:**
- Initialise a `DataContext` with ML datasets and metadata from the titanic use case
- Build some out-of-the-box validations on a trained model and the `DataContext` with the `ModelValidator`:
    - Minimum functionality validation
    - Performance validations:
        - With sklearn metrics
        - With custom metrics
        - On specific data sclices
    - Explainability validations (with permutation importance)
    - Inference time validation
- Build a custom validation
- Save validations to a `Trubric`
- Execute the `Trubric` from file
- Execute the `Trubric` from the CLI tool

## Load data & model

In [ ]:
from trubrics.example import get_titanic_data_and_model
train_df, test_df, model = get_titanic_data_and_model()

## Init DataContext

In [ ]:
from trubrics.context import DataContext

In [ ]:
data_context = DataContext(
    name="my_first_dataset",
    version=0.1,
    testing_data=test_df,
    target="Survived",
    training_data=train_df,
    minimum_functionality_data=test_df.head(),
)

## Init ModelValidator

In [ ]:
from trubrics.validations import ModelValidator
from examples.classification_titanic.custom_scorer import custom_scorers  # see .py script for custom_scorer example
from examples.classification_titanic.slicing_functions import slicing_functions  # see .py script for slicing_functions examples

In [ ]:
model_validator = ModelValidator(
    data=data_context, model=model, custom_scorers=custom_scorers, slicing_functions=slicing_functions
)

## Use the ModelValidator to build out-of-the-box validations

In [ ]:
import rich
minimum_functionality = [
    model_validator.validate_minimum_functionality(severity="warning"),  # validation severity can be passed in to any validation
]
_ = [rich.print(val.dict()) for val in minimum_functionality]

In [ ]:
import sklearn.metrics
rich.print("View sklearn default scorers: ", sklearn.metrics.SCORERS)

In [ ]:
performance = [
    model_validator.validate_performance_against_threshold(metric="accuracy", threshold=0.8, data_slice="children"),
    model_validator.validate_performance_against_threshold(metric="accuracy", threshold=0.8, data_slice="male"),
    model_validator.validate_performance_against_threshold(metric="recall", threshold=0.7, severity="warning"),
    model_validator.validate_performance_against_threshold(metric="precision", threshold=0.7, severity="error"),
    model_validator.validate_performance_against_threshold(metric="f1", threshold=0.7, severity="experiment"),
    model_validator.validate_performance_against_threshold(metric="my_custom_loss", threshold=-0.7, severity="experiment"),
    model_validator.validate_performance_between_train_and_test(metric="recall", threshold=0.3, data_slice="male"),
    model_validator.validate_performance_between_train_and_test(metric="my_custom_loss", threshold=0.2),
    model_validator.validate_test_performance_against_dummy(metric="accuracy"),
    model_validator.validate_test_performance_against_dummy(metric="accuracy", strategy="constant", dummy_kwargs={"constant": 0}, data_slice="male"),
    model_validator.validate_performance_std_across_slices(metric="accuracy", dataset="testing_data", data_slices=["male", "children"], std_threshold=0.07),
    model_validator.validate_performance_std_across_slices(metric="recall", dataset="training_data", data_slices=["male", "female"], std_threshold=0.05, include_global_performance=True)
]
_ = [rich.print(val.dict()) for val in performance]

In [ ]:
import pandas as pd
# recap of calculated model performances by validations
pd.DataFrame(model_validator.performances)

In [ ]:
explainability = [
    model_validator.validate_feature_in_top_n_important_features(dataset="testing_data", feature="Sex", top_n_features=3),
    model_validator.validate_feature_in_top_n_important_features(dataset="training_data", feature="Age", top_n_features=2, permutation_kwargs={"n_repeats": 1, "random_state": 88, "n_jobs": -1}),
    model_validator.validate_feature_importance_between_train_and_test(),
    model_validator.validate_feature_importance_between_train_and_test(top_n_features=1),
]
_ = [rich.print(val.dict()) for val in explainability]

In [ ]:
inference_time = [
    model_validator.validate_inference_time(threshold=0.04, n_executions=100)
]
_ = [rich.print(val.dict()) for val in inference_time]

## Build custom validations

In [ ]:
from examples.classification_titanic.custom_validator import CustomValidator  # see script for CustomValidator example

In [ ]:
model_custom_validator = CustomValidator(data=data_context, model=model, custom_scorers=custom_scorers)

In [ ]:
custom = [model_custom_validator.validate_master_age(age_limit_master=13, severity="warning")]
_ = [rich.print(val.dict()) for val in custom]

## Save validations as a trubric

In [ ]:
from trubrics.validations import Trubric

validations = minimum_functionality + performance + explainability + inference_time + custom

trubric = Trubric(
    name="my_first_trubric",
    model_name="my_model",
    data_context_name=data_context.name,
    data_context_version=data_context.version,
    metadata={"tag": "master"},
    validations=validations,
)

In [ ]:
# save trubric to a local .json
trubric.save_local(path=".")

In [ ]:
# or save to trubrics UI
user_id = None  # enter User ID from trubrics manager here
url = None  # enter api url for trubrics manager here

try:
    trubric.save_ui(url=url, user_id=user_id)
    print("Trubric saved to UI.")
except Exception as e:
    print(f"Error in saving to trubrics manager:\n{e}")

## Execute the trubric from file

In [ ]:
from trubrics.validations.run import TrubricRun, run_trubric

In [ ]:
trubric_run_context = TrubricRun(
    data_context=data_context,
    model=model,
    trubric=Trubric.parse_file("./my_first_trubric.json"),
    custom_validator=CustomValidator,
    custom_scorers=custom_scorers,
    slicing_functions=slicing_functions
)
all_validation_results = run_trubric(trubric_run_context)

In [ ]:
for validation_result in all_validation_results:
    rich.print(f"{validation_result.validation_type} - {validation_result.severity.upper()}.......{validation_result.outcome}")

## Execute the trubric from the CLI tool

In [ ]:
!trubrics run \
    --no-save-ui \
    --trubric-config-path "." \
    --trubric-output-file-path "." \
    --trubric-output-file-name "my_new_trubric.json"